#Differentiating Flower With CNNs with TensorFlow

##Importing Packages

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import urllib
import cv2
import os
from tensorflow.keras import layers
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

##Loading The Dataset

In [0]:
splits = ['train[:70%]','train[70%:]']

(training_set, validation_set), dataset_info = tfds.load('tf_flowers',
                                                         split=splits,
                                                         with_info=True,
                                                         as_supervised=True)
num_training_examples=dataset_info.splits['train[70%:]'].num_examples
num_validation_examples=dataset_info.splits['train[:70%]'].num_examples
num_classes=dataset_info.features['label'].num_classes

In [0]:
print('Total Number of Classes: {}'.format(num_classes))
print('Total Number of Training Images: {}'.format(num_training_examples))
print('Total Number of Validation Images: {} \n'.format(num_validation_examples))
type(training_set)

In [0]:
for i, example in enumerate(training_set.take(5)):
  print('Image {} shape: {} label: {}'.format(i+1, example[0].shape, example[1]))

In [0]:
IMAGE_RES = 224

def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label

BATCH_SIZE = 32

train_batches = training_set.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)

validation_batches = validation_set.map(format_image).batch(BATCH_SIZE).prefetch(1)

##Building Model

In [0]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES, 3))

In [0]:
feature_extractor.trainable=False

In [0]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(num_classes)
])

model.summary()

##Training the Model

In [0]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

EPOCHS = 6

history = model.fit(train_batches,
                    epochs=6,
                    validation_data = validation_batches
                    )

##Predictions

In [0]:
class_names = np.array(dataset_info.features['label'].names)

print(class_names)


In [0]:
image_batch, label_batch = next(iter(train_batches))


image_batch = image_batch.numpy()
label_batch = label_batch.numpy()



predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()

predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]

print(predicted_class_names)


In [0]:
print("Labels:           ", label_batch)
print("Predicted labels: ", predicted_ids)

##Result

In [0]:
plt.figure(figsize=(10,9))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.subplots_adjust(hspace = 0.3)
  plt.imshow(image_batch[n])
  color = "blue" if predicted_ids[n] == label_batch[n] else "red"
  plt.title(predicted_class_names[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")

##Predicting single image

In [0]:
Path='./rose/daisy.jpeg'
imagess=cv2.imread(Path)
imagess=cv2.cvtColor(imagess, cv2.COLOR_BGR2RGB)
imagess=cv2.resize(imagess,(224,224))
labelss='daisy'
plt.imshow(imagess)
imagess = np.expand_dims(imagess, 0)
plt.title(labelss.title())
result = model.predict(imagess)
predicted = tf.squeeze(result).numpy()
predicted_id = np.argmax(predicted, axis=-1)
predicted_class_namess=class_names[predicted_id]
color = "white" if predicted_class_namess == labelss else "red"
plt.title(predicted_class_namess.title(),color=color,fontsize=50)
plt.axis('off')
plt.show()